In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import random
import time
import re

# Base URLs for each category
base_urls = {
    "CapHit": "https://www.spotrac.com/nfl/rankings/player/_/year/{}/sort/cap_total",
    "ContractAverage": "https://www.spotrac.com/nfl/rankings/player/_/year/{}/sort/contract_average",
    "ContractLength": "https://www.spotrac.com/nfl/rankings/player/_/year/{}/sort/contract_length"
}

# List of years to scrape
years = range(2011, 2025)

# User-Agent pool for rotation
user_agents = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.5060.134 Safari/537.36",
]

# Dictionary to store all scraped data
all_data = {category: [] for category in base_urls.keys()}

# Function to clean numeric values
def clean_numeric(value, is_currency=False):
    """
    Extracts the numeric portion of a string that starts with a dollar sign ($), 
    ignoring any text before the dollar sign.
    """
    # Search for a dollar sign followed by the numeric value
    match = re.search(r'\$(\d[\d,]*(?:\.\d{1,2})?)', value)
    if match:
        # Extract the numeric portion
        cleaned = match.group(1)
        numeric_value = float(cleaned.replace(',', ''))
        return int(numeric_value) if not is_currency else numeric_value
    return None

# Function to scrape a single category for a single year
def scrape_category(year, category, url):
    print(f"Scraping {category} data for the year {year}...")
    headers = {"User-Agent": random.choice(user_agents)}
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')

        # Initialize lists for this category and year
        ranks = []
        players = []
        teams = []
        positions = []
        values = []

        # Locate the list-group containing the players
        players_list = soup.find_all('li', class_='list-group-item')

        # Extract data
        for player in players_list:
            try:
                # Rank
                rank = player.find('div', class_='fw-bold').text.strip()
                
                # Player Name
                player_name = player.find('div', class_='link').text.strip()
                
                # Team and Position
                team_pos = player.find('small').text.strip()
                team, position = map(str.strip, team_pos.split(','))
                
                # Value (Cap Hit, Contract Average, or Contract Length)
                raw_value = player.find('span', class_='medium').text.strip()
                if category in ['CapHit', 'ContractAverage']:
                    value = clean_numeric(raw_value, is_currency=True)  # Keep currency formatting
                elif category == 'ContractLength':
                    value = clean_numeric(raw_value, is_currency=False)  # Strip currency formatting
                    
                # Append to lists
                ranks.append(rank)
                players.append(player_name)
                teams.append(team)
                positions.append(position)
                values.append(value)
            except AttributeError:
                # Skip problematic rows (e.g., banners)
                continue

        # Create a DataFrame for this category and year
        df = pd.DataFrame({
            'Season': [year] * len(ranks),
            'Rank': ranks,
            'Player': players,
            'Team': teams,
            'Position': positions,
            category: values
        })

        # Convert Rank to numeric and handle ties
        df['Rank'] = pd.to_numeric(df['Rank'], errors='coerce')
        df['Rank'] = df['Rank'].ffill()

        # Return the DataFrame
        return df

    except requests.exceptions.RequestException as e:
        print(f"Failed to scrape {category} data for {year}: {e}")
        return None

# Loop through each year and category
for year in years:
    for category, base_url in base_urls.items():
        # Construct the URL for this category and year
        url = base_url.format(year)

        # Scrape the data
        df = scrape_category(year, category, url)
        if df is not None:
            all_data[category].append(df)

        # Introduce a random delay between requests
        delay = random.uniform(5, 15)  # Delay between 5 and 15 seconds
        print(f"Delaying for {delay:.2f} seconds to avoid detection...")
        time.sleep(delay)

# Combine all data for each category into a single DataFrame
final_dataframes = {category: pd.concat(all_dfs, ignore_index=True) for category, all_dfs in all_data.items()}

# Save all data to an Excel file with separate sheets
output_file = "NFL_Salary_Rankings_2011_2024.xlsx"
with pd.ExcelWriter(output_file, engine='xlsxwriter') as writer:
    for category, df in final_dataframes.items():
        # Write data to sheet
        df.to_excel(writer, index=False, sheet_name=category)

        # Apply formatting
        workbook = writer.book
        worksheet = writer.sheets[category]
        if category in ['CapHit', 'ContractAverage']:
            # Format currency columns
            currency_format = workbook.add_format({'num_format': '$#,##0', 'align': 'left'})
            worksheet.set_column(df.columns.get_loc(category), df.columns.get_loc(category), 15, currency_format)
        elif category == 'ContractLength':
            # Format number columns
            number_format = workbook.add_format({'num_format': '0', 'align': 'left'})
            worksheet.set_column(df.columns.get_loc(category), df.columns.get_loc(category), 15, number_format)

print(f"Data has been scraped and saved to {output_file} with proper formatting.")

Scraping CapHit data for the year 2011...
Delaying for 6.23 seconds to avoid detection...
Scraping ContractAverage data for the year 2011...
Delaying for 6.34 seconds to avoid detection...
Scraping ContractLength data for the year 2011...
Delaying for 9.84 seconds to avoid detection...
Scraping CapHit data for the year 2012...
Delaying for 7.45 seconds to avoid detection...
Scraping ContractAverage data for the year 2012...
Delaying for 11.19 seconds to avoid detection...
Scraping ContractLength data for the year 2012...
Delaying for 5.57 seconds to avoid detection...
Scraping CapHit data for the year 2013...
Delaying for 8.54 seconds to avoid detection...
Scraping ContractAverage data for the year 2013...
Delaying for 14.69 seconds to avoid detection...
Scraping ContractLength data for the year 2013...
Delaying for 8.18 seconds to avoid detection...
Scraping CapHit data for the year 2014...
Delaying for 13.20 seconds to avoid detection...
Scraping ContractAverage data for the year 201